In [91]:
import pandas as pd
train_data = pd.read_csv("candy-data.csv", delimiter=',' , index_col='competitorname')
#train_data.competitorname.tolist() -- список названий

In [92]:
badnames = ['Fun Dip','Laffy Taffy', 'Peanut butter M&Ms']
train_data = train_data.drop(badnames)
train_data.head()

,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent,Y
competitorname,,,,,,,,,,,,,
100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725,1
3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936,1
One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086,0
One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505,0
Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465,1


In [93]:
X = train_data.drop(['winpercent', 'Y'], axis=1)
y = train_data['Y']

In [94]:
from sklearn.linear_model import LogisticRegression

In [95]:
clf = LogisticRegression(random_state=2019, solver='lbfgs')

In [96]:
clf.fit(X, y)

LogisticRegression(random_state=2019)

In [97]:
test_data = pd.read_csv("candy-test.csv", delimiter=',', index_col='competitorname')
test_data.head()

,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,Y
competitorname,,,,,,,,,,,,
Sugar Babies,0,1,1,0,0,0,1,0,1,0.647364,0.7670,0
Sugar Daddy,1,0,0,0,1,0,1,0,0,0.418000,0.3250,0
Super Bubble,0,1,0,0,0,0,0,0,0,0.162000,0.1160,0
Swedish Fish,0,1,0,0,0,0,0,0,1,0.604000,0.7550,1
Tootsie Pop,1,0,0,0,0,0,0,0,0,0.876560,0.5654,1


In [98]:
X_test = test_data.drop(['Y'], axis=1)
Y_pred = reg.predict(X_test)
Y_pred

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1])

In [99]:
Y_true = (test_data['Y'].to_frame().T).values.ravel()
Y_true

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1])

In [100]:
test_data['winpercent'] = reg.predict_proba(X_test).tolist()
test_data

,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,Y,winpercent
competitorname,,,,,,,,,,,,,
Sugar Babies,0,1,1,0,0,0,1,0,1,0.647364,0.76700,0,"[0.7183079127807949, 0.28169208721920513]"
Sugar Daddy,1,0,0,0,1,0,1,0,0,0.418000,0.32500,0,"[0.6206097960858793, 0.3793902039141207]"
Super Bubble,0,1,0,0,0,0,0,0,0,0.162000,0.11600,0,"[0.6019512128334747, 0.39804878716652525]"
Swedish Fish,0,1,0,0,0,0,0,0,1,0.604000,0.75500,1,"[0.6151787906408426, 0.38482120935915737]"
Tootsie Pop,1,0,0,0,0,0,0,0,0,0.876560,0.56540,1,"[0.23818987468460173, 0.7618101253153983]"
Tootsie Roll Juniors,1,0,0,0,0,0,0,0,0,0.313000,0.51100,0,"[0.2962036612105454, 0.7037963387894546]"
Tootsie Roll Midgies,1,0,0,0,0,0,0,0,1,0.174000,0.01100,0,"[0.34166511091138174, 0.6583348890886183]"
Tootsie Roll Snack Bars,1,0,0,0,0,0,0,1,0,0.465000,0.32500,1,"[0.20891080787416383, 0.7910891921258362]"
Trolli Sour Bites,0,1,0,0,0,0,0,0,1,0.313000,0.25500,0,"[0.6355719184490793, 0.36442808155092066]"


In [101]:
Y_pred_probs_class_1 = Y_pred_probs[:, 1]

In [102]:
#подключение библиотеки для вычисления метрик
from sklearn import metrics

In [103]:
fpr, tpr, _ = metrics.roc_curve(Y_true, Y_pred)

In [104]:
metrics.roc_auc_score(Y_true, Y_pred_probs_class_1)

0.8400000000000001

In [105]:
metrics.recall_score(Y_true, Y_pred)

0.6

In [106]:

metrics.precision_score(Y_true, Y_pred)

0.5